<a href="https://colab.research.google.com/github/Ghisbe/Alarmas_pictogramas/blob/main/pre_procesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preprocesamiento columna 'description'.:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import gdown

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url = "https://drive.google.com/uc?id=1MdxRAfh0ojpqayiSGiieg_S9Zo0vdjGt"


In [ ]:
output = 'df_definitivo.csv'
gdown.download(url, output, quiet=False)

# Leer el archivo CSV en un DataFrame de pandas
df_alertas = pd.read_csv(output)

Downloading...
From (original): https://drive.google.com/uc?id=1MdxRAfh0ojpqayiSGiieg_S9Zo0vdjGt
From (redirected): https://drive.google.com/uc?id=1MdxRAfh0ojpqayiSGiieg_S9Zo0vdjGt&confirm=t&uuid=dd787ed4-0cd0-4373-aeb8-c1f1b042c773
To: /content/df_definitivo.csv
100%|██████████| 1.51G/1.51G [00:15<00:00, 94.3MB/s]


In [ ]:
df_alertas

,id,event,description,instruction,urgency,severity
0,5ed3a4b2283336e2e7db3a1c,Special Weather Statement,"At 630 AM MDT/530 AM PDT/, Doppler radar was t...",Frequent cloud to ground lightning is occurrin...,Expected,Moderate
1,5ed3a07c283336e2e7d2bfa4,Flood Warning,...The National Weather Service in Billings MT...,Safety message...A river Flood Warning means r...,Expected,Severe
2,5ed39e54283336e2e7d22306,Flood Advisory,"At 502 AM PDT, Gauge reports continue to indic...",A Flood Advisory means river or stream flows a...,Expected,Minor
3,5ed39e11283336e2e7d15e23,Flood Watch,The National Weather Service in Spokane has is...,A Flood Watch means there is a potential for f...,Future,Severe
4,5ed3962c283336e2e7c1d935,Flood Warning,The Flood Warning has been cancelled and is no...,Safety message...Motorists should not attempt ...,Past,Minor
...,...,...,...,...,...,...
1497255,613a5aee434e34d5ee03f0e1,Special Weather Statement,"At 814 AM CDT, Doppler radar was tracking a st...","If outdoors, consider seeking shelter inside a...",Expected,Moderate
1497256,613a5c5b434e34d5ee06813a,Special Weather Statement,"At 816 AM CDT, Doppler radar was tracking a st...","If outdoors, consider seeking shelter inside a...",Expected,Moderate
1497257,613a5c5b434e34d5ee06813f,Flash Flood Watch,The National Weather Service in El Paso Tx/San...,You should monitor later forecasts and be prep...,Future,Severe
1497258,613a5d4d434e34d5ee06a9a9,Dense Fog Advisory,* WHAT...Visibility around 1/4 mile or less in...,"If driving, slow down, use your headlights, an...",Expected,Moderate


Se toma la decision de usar el 50% del dataset

In [ ]:
# Seleccionar la primera mitad del dataset
alertas_nuevo = df_alertas.iloc[:len(df_alertas)//2]

In [ ]:
#alertas_nuevo.to_csv('alertas.csv', index=False)

# Informacion General

In [ ]:
alertas_nuevo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748630 entries, 0 to 748629
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           748630 non-null  object
 1   event        748630 non-null  object
 2   description  748630 non-null  object
 3   instruction  748630 non-null  object
 4   urgency      748630 non-null  object
 5   severity     748630 non-null  object
dtypes: object(6)
memory usage: 34.3+ MB


###Pre procesamiento de columna 'description".

1. Eliminación de ruido:
Este paso tiene como objetivo eliminar todos aquellos símbolos o caracteres que no aportan nada en el significado de las frases.

Pre procesamiento de columna 'description".

In [ ]:
#Observamos una muestra al azar de 15 ejemplos para visualizar la columna a analizar.
(alertas_nuevo['description']).sample(15)

389984    * LOCATIONS...Livingston, Ontario, Cattaraugus...
369155    A prolonged snow event is expected to impact t...
142467    The National Weather Service in Chicago has is...
630542    The National Weather Service in Nashville has ...
459660    The Flood Warning continues for\nThe Christina...
540502    At 451 PM CDT, Doppler radar was tracking a st...
125558    The National Weather Service in Flagstaff has ...
738788    At 1257 AM CDT, Doppler radar was tracking str...
123668    ...AREAS OF MARINE DENSE FOG CONTINUE ACROSS C...
208086    At 755 PM CDT, a severe thunderstorm was locat...
588680    At 316 PM MDT, National Weather Service Dopple...
491400    .VERY COLD TEMPERATURES COMBINED WITH GUSTY WI...
82118     ...SNOW ON SUNDAY...\n\n.Light to moderate sno...
269511    An approaching upper level storm system will i...
195747    At 425 PM EDT, a severe thunderstorm was locat...
Name: description, dtype: object

Vamos a utilizar la fila 142406
 ya que ahi observamos la presencia de las palabras \n\n y WHAT\WHERE asi que analizamos todo el texto de esa fila, haciendo un split de todas las palabras separadas por espacio.

In [ ]:
fila = (alertas_nuevo['description'].iloc[142406]).split()
print(fila)

['*', 'WHAT...North', 'to', 'northeast', 'winds', '15', 'to', '25', 'mph', 'with', 'gusts', 'to', '40', 'mph.', '*', 'WHERE...Western', 'San', 'Fernando', 'Valley.', '*', 'WHEN...Until', '3', 'PM', 'PDT', 'Thursday.', '*', 'IMPACTS...Gusty', 'winds', 'will', 'make', 'driving', 'difficult', 'and', 'could', 'blow', 'around', 'unsecured', 'objects.', 'Tree', 'limbs', 'could', 'be', 'blown', 'down.', '*', 'ADDITIONAL', 'DETAILS...']


Observamos que las palabras 'WHAT', 'WHERE', 'WHEN' e 'IMPACTS' estan unidas a la segunda palabra por tres puntos '...', por ello vamos a reemplazar por un espacio para separar las palabras. Despues se trabajara con puntos, asteriscos y demas signos que no sean relevantes al modelo (Normalizacion).

In [ ]:
alertas_nuevo['description'] = alertas_nuevo['description'].str.replace('...', ' ', regex=False)
alertas_nuevo['description'].iloc[142406] #Probamos esa fila

<ipython-input-9-c85d9a838523>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alertas_nuevo['description'] = alertas_nuevo['description'].str.replace('...', ' ', regex=False)


'* WHAT North to northeast winds 15 to 25 mph with gusts to 40\nmph.\n\n* WHERE Western San Fernando Valley.\n\n* WHEN Until 3 PM PDT Thursday.\n\n* IMPACTS Gusty winds will make driving difficult and could\nblow around unsecured objects. Tree limbs could be blown down.\n\n* ADDITIONAL DETAILS '

Ahora tambien observamos que figura como textos codigos de salto de linea "\n\n" y "\n".

In [ ]:
#Se observan palabras con "\n"
palabra_a_contar1 = '\n'
palabras_1 = alertas_nuevo['description'].str.count(palabra_a_contar1).sum()
print(palabras_1)

12277036


Es una cantidad importe de esas palabras. En este caso reemplazamos como el caso anterior por un espacio.

In [ ]:
df_alertas['description'] = df_alertas['description'].str.replace('\n', ' ', regex=False)
df_alertas['description'].iloc[142406] #Probamos esa fila

'* WHAT...North to northeast winds 15 to 25 mph with gusts to 40 mph.  * WHERE...Western San Fernando Valley.  * WHEN...Until 3 PM PDT Thursday.  * IMPACTS...Gusty winds will make driving difficult and could blow around unsecured objects. Tree limbs could be blown down.  * ADDITIONAL DETAILS...'

Observamos una muestra de 10 filas para observar estos cambios:

In [ ]:
df_alertas.sample(10)

,id,event,description,instruction,urgency,severity
126259,5e6db2a9023f1930efb684af,Marine Weather Statement,The areas affected include... Biscayne Bay......,Mariners can expect gusty winds to around 30 k...,Expected,Minor
13535,5ec2fab0283336e2e702e758,Flood Warning,...The National Weather Service in Northern In...,Never drive vehicles through flooded areas. Th...,Expected,Severe
1309469,5f6fb86131a8c6681ca35f59,Wind Advisory,The Wind Advisory has been cancelled and is no...,"Use extra caution when driving, especially if ...",Past,Minor
414915,5e6dbb5c023f1930efe181a0,Flood Warning,Lower Roanoke River At Roanoke Rapids affectin...,PRECAUTIONARY/PREPAREDNESS ACTIONS...\n\nSafet...,Immediate,Severe
552665,5e6dbe33023f1930efef178b,Severe Weather Statement,AT 828 PM CDT...A SEVERE THUNDERSTORM CAPABLE ...,HEAVY RAINFALL MAY HIDE THIS TORNADO. DO NOT W...,Immediate,Extreme
1116289,5e6dcb66023f1930ef2a73e6,Flood Warning,"* At 555 AM EDT, Doppler radar and automated r...",Most flood deaths occur in vehicles. Do not at...,Immediate,Severe
88796,5e6db1f8023f1930efb2ed6f,Winter Weather Advisory,* WHAT...Snow...moderate to heavy at times tod...,Slow down and use caution while traveling.\n\n...,Expected,Moderate
1096670,5e6dcad7023f1930ef2808b6,Special Weather Statement,"At 754 PM PDT, Doppler radar was tracking a st...",Brief torrential rainfall is also occurring wi...,Expected,Moderate
411519,5e6dd97b023f1930ef63dc8a,Flood Warning,Altamaha River At Baxley affecting Appling...T...,PRECAUTIONARY/PREPAREDNESS ACTIONS...\n\nAll p...,Immediate,Severe
333227,5e6ddb34023f1930ef6a4a8a,Special Weather Statement,"At 907 PM CDT, Meteorologists were tracking a ...",High winds can bring down trees and large limb...,Expected,Moderate


###Normalización:
La normalización es una tarea que tiene como objetivo poner todo el texto en igualdad de condiciones:
- Convertir todo el texto en mayúscula o minúsculas
- Eliminar, puntos, comas, comillas, etc.
- Convertir los números a su equivalente a palabras
- Quitar las Stop-words
Empezamos con pasar todo el texto a minuscula:




In [ ]:
df_alertas['description'] = df_alertas['description'].str.lower()
df_alertas['description'].iloc[142406] #Probamos esa fila

'* what...north to northeast winds 15 to 25 mph with gusts to 40 mph.  * where...western san fernando valley.  * when...until 3 pm pdt thursday.  * impacts...gusty winds will make driving difficult and could blow around unsecured objects. tree limbs could be blown down.  * additional details...'

Eliminar, puntos, comas, comillas, etc. Para ello utilizamos expresiones regulares.

In [ ]:
import re

# Eliminar puntuación usando expresiones regulares
df_alertas['description'] = df_alertas['description'].str.replace(r'[^\w\s]', '', regex=True)

df_alertas['description'].iloc[142406] #Probamos esa fila

' whatnorth to northeast winds 15 to 25 mph with gusts to 40 mph   wherewestern san fernando valley   whenuntil 3 pm pdt thursday   impactsgusty winds will make driving difficult and could blow around unsecured objects tree limbs could be blown down   additional details'

Pasando numeros a letras. Usamos la libreria num2words

In [ ]:
!pip install num2words


In [ ]:
from num2words import num2words

# Función para extraer números de una cadena de texto y convertirlos en palabras
def convert_numbers_to_words(text):
    numbers = re.findall(r'\d+', text) #Busca numeros
    for number in numbers:
        text = text.replace(number, num2words(int(number)))
    return text

df_alertas['description'] = df_alertas['description'].apply(convert_numbers_to_words)

df_alertas['description'].iloc[142406] #Probamos esa fila

' whatnorth to northeast winds fifteen to twenty-five mph with gusts to forty mph   wherewestern san fernando valley   whenuntil three pm pdt thursday   impactsgusty winds will make driving difficult and could blow around unsecured objects tree limbs could be blown down   additional details'

###Tokenización:

Este paso tiene como objetivo dividir las cadenas de texto del documento en piezas más pequeñas o tokens.
Aunque la tokenización es el proceso de dividir grandes cadenas de texto en cadenas más pequeñas, se suele diferenciar la:
- Segmentation: Tarea de dividir grandes cadenas de texto en piezas más pequeñas como oraciones o párrafos.
- Tokenization: Tarea de dividir grandes cadenas de texto solo y exclusivamente en palabras.

Se va a usar la biblioteca NLTK.

In [ ]:
!pip install nltk

In [ ]:
import nltk

Se descarga el modelo pre entrenado 'punkt' para realizar la tokenizacion de manera efectiva.

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Se hace este proceso en lotes ya que es un extenso el dataset

In [ ]:
#tokenizar texto en lotes
def tokenizador(texts, batch_size=100):
    tokenized_texts = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        tokenized_batch = [nltk.word_tokenize(str(text)) for text in batch]
        tokenized_texts.extend(tokenized_batch)
    return tokenized_texts

# Tokenizar en lotes y aplicar a la columna "description"
batch_size = 100
df_alertas['description_token'] = tokenizador(df_alertas['description'], batch_size)

In [ ]:
df_alertas['description'].iloc[142406] #Probamos esa fila

In [ ]:
df_alertas['description_token']

Ahora las se elimina las stop words que son palabras que se filtran o eliminan del texto durante el procesamiento del lenguaje natural (NLP) porque se consideran comunes y no aportan un significado informativo específico para el análisis. Usamos la libreria spacy

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 69.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from nltk.corpus import stopwords
import spacy

In [ ]:
nltk.download('stopwords')

In [ ]:
nlp = spacy.load('en_core_web_sm')
# Configurar stop words en inglés
stop_words = set(stopwords.words('english'))
# Función para eliminar stop words de una lista de tokens
def remove_stop_words(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
df_alertas['description_no_stopwords'] = df_alertas['description_token'].apply(lambda x: remove_stop_words(x))

Lemantizacion

In [ ]:
def apply_lemmatization(tokens):
    doc = nlp(' '.join(tokens))
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)

# Aplicar la lematización a la columna 'description_no_stopwords'
df_alertas['description_lemmatized'] = df_alertas['description_no_stopwords'].apply(lambda x: apply_lemmatization(x))

# Mostrar algunas filas del DataFrame procesado
print(df_alertas[['description', 'description_no_stopwords', 'description_lemmatized']].head())